### LOAD PACKAGES 

In [36]:
# Importation des librairies 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report,roc_auc_score,precision_score,recall_score, precision_recall_curve, auc, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization



pd.set_option('display.max_columns', 100)

In [37]:
%run "Feature engineering.ipynb" import *

### READ THE DATA 

In [38]:
# # # Lire la data stockée dans le fichier CSV. Modifier le chemin si vous relancer le kernel 
# # #df=pd.read_csv(r'C:\Users\anas.sory\Documents\Stage\creditcard.csv')
# train_df=pd.read_csv(r'C:\Users\MohamedAliELBOURI\Desktop\Projet\fraude-main\fraude-main\Data\archive fraude CB7\fraudTrain.csv')
# test_df=pd.read_csv(r'C:\Users\MohamedAliELBOURI\Desktop\Projet\fraude-main\fraude-main\Data\archive fraude CB7\fraudTest.csv')

### CHECK THE DATA 

### Preparing and splitting the data

In [39]:
# cols = train_df.columns.tolist()
# cols = [c for c in cols if c not in ["is_fraud"]]
# target = "is_fraud"
# print(cols)

# #Definition des nouvelles variables X_train and Y_train, X_val and Y_val
# X_train = train_df[cols]
# Y_train = train_df[target]
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, shuffle=False)

# #Definition des nouvelles variables X_test and Y_test
# X_test = test_df[cols]
# Y_test = test_df[target]

# print(X_train.shape)
# print(Y_train.shape)
# print(X_val.shape)
# print(Y_val.shape)
# print(X_test.shape)
# print(Y_test.shape)


In [40]:
def config1_1(X_train, X_test, cat_columns = ['category', 'state']):

    X_train_1_1, X_test_1_1 = X_train.copy(), X_test.copy()
    # We label encode 'merchant'
    le = LabelEncoder()
    le.fit(X_train_1_1['merchant'])
    X_train_1_1['merchant'] = le.transform(X_train_1_1['merchant'])
    X_test_1_1['merchant'] = le.transform(X_test_1_1['merchant'])

    # Instantiate encoder/scaler
    scaler = StandardScaler()
    ohe    = OneHotEncoder(sparse=False, handle_unknown = 'ignore')

    # We fit the transformers on the Train data
    num_columns = X_train_1_1.select_dtypes(include=np.number).columns.tolist()  
    scaler.fit(X_train_1_1[num_columns])
    ohe.fit(X_train_1_1[cat_columns])

    # We apply the transformers on the Train and Test data
    scaled_columns_train  = scaler.transform(X_train_1_1[num_columns]) 
    encoded_columns_train =    ohe.transform(X_train_1_1[cat_columns])

    scaled_columns_test  = scaler.transform(X_test_1_1[num_columns]) 
    encoded_columns_test =    ohe.transform(X_test_1_1[cat_columns])

    # Concatenate (Column-Bind) Processed Columns Back Together
    X_train_ohe_std = np.concatenate([scaled_columns_train, encoded_columns_train], axis=1)
    X_test_ohe_std = np.concatenate([scaled_columns_test, encoded_columns_test], axis=1)
    ohe_std_cols = np.concatenate([num_columns, ohe.get_feature_names()])

    #Transforming both numpy array to dataframes
    X_train_ohe_std = pd.DataFrame(X_train_ohe_std, index=X_train.index, columns=ohe_std_cols)
    X_test_ohe_std = pd.DataFrame(X_test_ohe_std, index=X_test.index, columns=ohe_std_cols)

    return X_train_ohe_std, X_test_ohe_std
  

In [41]:
def config1_2(X_train, X_test, cat_columns = ['category', 'state', 'merchant']):
    X_train_1_2, X_test_1_2 = X_train.copy(), X_test.copy()
    
    # Instantiate scaler/labelEncoder"
    scaler = StandardScaler()
    le = LabelEncoder()

    #We transform categorical variables to integers
    for cat in cat_columns:
        le.fit(X_train[cat])
        X_train_1_2[cat] = le.transform(X_train_1_2[cat])
        X_test_1_2[cat] = le.transform(X_test_1_2[cat])

    # We fit the scaler on all numerical features (transformed cat_columns are included)
    num_columns = X_train_1_2.select_dtypes(include=np.number).columns.tolist()  
    scaler.fit(X_train_1_2[num_columns])

    # We apply the transformers on the Train and Test data
    scaled_columns_train  = scaler.transform(X_train_1_2[num_columns]) 
    scaled_columns_test  = scaler.transform(X_test_1_2[num_columns]) 

    #Transforming both numpy array to dataframes
    X_train_le_std = pd.DataFrame(scaled_columns_train, index=X_train.index, columns=num_columns)
    X_test_le_std = pd.DataFrame(scaled_columns_test, index=X_test.index, columns=num_columns)

    return X_train_le_std, X_test_le_std

  

In [42]:
def config2_1(X_train, Y_train, X_test, cat_columns = ['category', 'state', 'merchant']):
    # We define the numerical columns (transformed cat_columns are not included)
    num_columns = X_train.select_dtypes(include=np.number).columns.tolist()  

    X_train_2_1, X_test_2_1 = X_train.copy(), X_test.copy()
    for feature in cat_columns:
        categories, replace_list = fraud_risk_percentile_mapping(X_train, Y_train, feature)
        X_train_2_1[feature] = X_train_2_1[feature].replace(categories,replace_list)
        X_test_2_1[feature] = X_test_2_1[feature].replace(categories,replace_list)


    # Instantiate encoder/scaler
    scaler = StandardScaler()
    ohe    = OneHotEncoder(sparse=False, handle_unknown='ignore')

    # We fit the transformers on the Train data
    scaler.fit(X_train_2_1[num_columns])
    ohe.fit(X_train_2_1[cat_columns])

    # We apply the transformers on the Train and Test data
    scaled_columns_train  = scaler.transform(X_train_2_1[num_columns]) 
    encoded_columns_train =    ohe.transform(X_train_2_1[cat_columns])

    scaled_columns_test  = scaler.transform(X_test_2_1[num_columns]) 
    encoded_columns_test =    ohe.transform(X_test_2_1[cat_columns])

    # Concatenate (Column-Bind) Processed Columns Back Together
    X_train_ohe_std = np.concatenate([scaled_columns_train, encoded_columns_train], axis=1)
    X_test_ohe_std = np.concatenate([scaled_columns_test, encoded_columns_test], axis=1)
    ohe_std_cols = np.concatenate([num_columns, ohe.get_feature_names()])

    #Transforming both numpy array to dataframes
    X_train_ohe_std = pd.DataFrame(X_train_ohe_std, index=X_train.index, columns=ohe_std_cols)
    X_test_ohe_std = pd.DataFrame(X_test_ohe_std, index=X_test.index, columns=ohe_std_cols)

    return X_train_ohe_std, X_test_ohe_std
# X_train_2_1, X_val_2_1 = config2_1(X_train, Y_train, X_val)

In [43]:
def config2_2(X_train, Y_train, X_test, cat_columns = ['category', 'state', 'merchant']):
    X_train_2_1, X_test_2_1 = X_train.copy(), X_test.copy()
    for feature in cat_columns:
        categories, replace_list = fraud_risk_percentile_mapping(X_train, Y_train, feature)
        X_train_2_1[feature] = X_train_2_1[feature].replace(categories,replace_list)
        X_test_2_1[feature] = X_test_2_1[feature].replace(categories,replace_list)

    # Instantiate scaler
    scaler = StandardScaler()

    # We fit the scaler on all numerical features (transformed cat_columns are included)
    num_columns = X_train_2_1.select_dtypes(include=np.number).columns.tolist()  
    scaler.fit(X_train_2_1[num_columns])

    # We apply the transformers on the Train and Test data
    scaled_columns_train  = scaler.transform(X_train_2_1[num_columns]) 
    scaled_columns_test  = scaler.transform(X_test_2_1[num_columns]) 

    #Transforming both numpy array to dataframes
    X_train_le_std = pd.DataFrame(scaled_columns_train, index=X_train.index, columns=num_columns)
    X_test_le_std = pd.DataFrame(scaled_columns_test, index=X_test.index, columns=num_columns)

    return X_train_le_std, X_test_le_std
    
# X_train_2_2, X_val_2_2 = config2_2(X_train, Y_train, X_val)

In [44]:
def config3_1(X_train, X_test, cat_columns = ['category', 'state']):
    X_train_3_1, X_test_3_1 = X_train.copy(), X_test.copy()
    # We label encode 'merchant'
    le = LabelEncoder()
    le.fit(X_train_3_1['merchant'])
    X_train_3_1['merchant'] = le.transform(X_train_3_1['merchant'])
    X_test_3_1['merchant'] = le.transform(X_test_3_1['merchant'])

    # We define the numerical columns (including merchant)
    num_columns = X_train_3_1.select_dtypes(include=np.number).columns.tolist()  

    # We transform 'category' to label encoding (ordered by necessity of the transaction)
    X_train_3_1['category'], X_test_3_1['category'] = X_transform_transaction_category(X_train, X_test)

    # We transform states to regions (51 states to 5 geographic regions)
    states, regions = state_to_region(X_train_3_1)
    X_train_3_1['state'] = X_train_3_1['state'].replace(states, regions)
    X_test_3_1['state'] = X_test_3_1['state'].replace(states, regions)

    # Instantiate scaler/encoder
    scaler = StandardScaler()
    ohe    = OneHotEncoder(sparse=False)

    # We fit the transformers on the Train data
    scaler.fit(X_train_3_1[num_columns])
    ohe.fit(X_train_3_1[cat_columns])

    # We apply the transformers on the Train and Test data
    scaled_columns_train  = scaler.transform(X_train_3_1[num_columns]) 
    encoded_columns_train =    ohe.transform(X_train_3_1[cat_columns])

    scaled_columns_test  = scaler.transform(X_test_3_1[num_columns]) 
    encoded_columns_test =    ohe.transform(X_test_3_1[cat_columns])

    # Concatenate (Column-Bind) Processed Columns Back Together
    X_train_ohe_std = np.concatenate([scaled_columns_train, encoded_columns_train], axis=1)
    X_test_ohe_std = np.concatenate([scaled_columns_test, encoded_columns_test], axis=1)
    ohe_std_cols = np.concatenate([num_columns, ohe.get_feature_names()])

    #Transforming both numpy array to dataframes
    X_train_ohe_std = pd.DataFrame(X_train_ohe_std, index=X_train.index, columns=ohe_std_cols)
    X_test_ohe_std = pd.DataFrame(X_test_ohe_std, index=X_test.index, columns=ohe_std_cols)

    return X_train_ohe_std, X_test_ohe_std


In [45]:
def config3_2(X_train, X_test):
    X_train_3_2, X_test_3_2 = X_train.copy(), X_test.copy()
    # We label encode 'merchant'
    le = LabelEncoder()
    le.fit(X_train_3_2['merchant'])
    X_train_3_2['merchant'] = le.transform(X_train_3_2['merchant'])
    X_test_3_2['merchant'] = le.transform(X_test_3_2['merchant'])

    # We transform 'category' to label encoding (ordered by necessity of the transaction)
    X_train_3_2['category'], X_test_3_2['category'] = X_transform_transaction_category(X_train, X_test)

    # We transform states to regions (51 states to 5 geographic regions)
    states, regions = state_to_region(X_train_3_2)
    X_train_3_2['state'] = X_train_3_2['state'].replace(states, regions)
    X_test_3_2['state'] = X_test_3_2['state'].replace(states, regions)
    le.fit(X_train_3_2['state'])
    X_train_3_2['state'] = le.transform(X_train_3_2['state'])
    X_test_3_2['state'] = le.transform(X_test_3_2['state'])


    # We define the numerical columns (including merchant, state and category)
    num_columns = X_train_3_2.select_dtypes(include=np.number).columns.tolist()  


    # Instantiate scaler
    scaler = StandardScaler()

    # We fit the transformers on the Train data
    scaler.fit(X_train_3_2[num_columns])

    # We apply the transformers on the Train and Test data
    scaled_columns_train  = scaler.transform(X_train_3_2[num_columns]) 
    scaled_columns_test  = scaler.transform(X_test_3_2[num_columns]) 

    #Transforming both numpy array to dataframes
    X_train_std = pd.DataFrame(scaled_columns_train, index=X_train.index, columns=num_columns)
    X_test_std = pd.DataFrame(scaled_columns_test, index=X_test.index, columns=num_columns)

    return X_train_std, X_test_std

In [46]:
def dataTransformer(X_train, X_val = None, X_test = None, Y_train = None, config_name='config1_1'):
    # Defining copies for X
    X_train_copy = X_train.copy()
    X_val_copy = None
    X_test_copy = None
    if X_val is not None:
        X_val_copy = X_val.copy()    
    if X_test is not None:
        X_test_copy = X_test.copy()

    # Defining transformers:
    scaler = StandardScaler()
    ohe = OneHotEncoder()
    le = LabelEncoder()

    # Pre-processing for all configs:

    # Changing Gender to binary, and age to numerical
    X_train_copy['gender'] = X_train_copy['gender'].replace(['M','F'],[0,1])
    X_train_copy['dob'] = np.floor((pd.to_datetime(X_train_copy['trans_date_trans_time']) - pd.to_datetime(X_train_copy['dob']))/np.timedelta64(1,'Y'))
    if X_val_copy is not None:
        X_val_copy['gender'] = X_val_copy['gender'].replace(['M','F'],[0,1])
        X_val_copy['dob'] = np.floor((pd.to_datetime(X_val_copy['trans_date_trans_time']) - pd.to_datetime(X_val_copy['dob']))/np.timedelta64(1,'Y'))
    if X_test_copy is not None:
        X_test_copy['gender'] = X_test_copy['gender'].replace(['M','F'],[0,1])
        X_test_copy['dob'] = np.floor((pd.to_datetime(X_test_copy['trans_date_trans_time']) - pd.to_datetime(X_test_copy['dob']))/np.timedelta64(1,'Y'))


    # Removing Index column
    X_train_copy = X_train_copy.drop('Unnamed: 0',axis=1)
    if X_val_copy is not None:
        X_val_copy = X_val_copy.drop('Unnamed: 0',axis=1)
    if X_test_copy is not None:
        X_test_copy = X_test_copy.drop('Unnamed: 0',axis=1)



    # Removing Credit Card Number
    X_train_copy = X_train_copy.drop('cc_num',axis=1)
    if X_val_copy is not None:
        X_val_copy = X_val_copy.drop('cc_num',axis=1)
    if X_test_copy is not None:
        X_test_copy = X_test_copy.drop('cc_num',axis=1)

    # Removing trans_date_trans_time column
    X_train_copy = X_train_copy.drop('trans_date_trans_time',axis=1)
    if X_val_copy is not None:
        X_val_copy = X_val_copy.drop('trans_date_trans_time',axis=1)
    if X_test_copy is not None:
        X_test_copy = X_test_copy.drop('trans_date_trans_time',axis=1)
    
    # Removing trans_num column
    X_train_copy = X_train_copy.drop('trans_num',axis=1)
    if X_val_copy is not None:
        X_val_copy = X_val_copy.drop('trans_num',axis=1)
    if X_test_copy is not None:
        X_test_copy = X_test_copy.drop('trans_num',axis=1)



    # No feature selection, no pre-preprocessing, labelEncoding categorical  variables and standard scaling numerical variables
    if config_name == "config0": 
        # Defining numerical and categorical features
        num_columns = X_train_copy.select_dtypes(include=np.number).columns.tolist() 
        cat_columns = X_train_copy.select_dtypes(include=np.object).columns.tolist()   
        # fitting the scaler
        scaler.fit(X_train_copy[num_columns])
        X_train_copy[num_columns] = scaler.transform(X_train_copy[num_columns])
        if X_val_copy is not None:
            X_val_copy[num_columns] = scaler.transform(X_val_copy[num_columns])
        if X_test_copy is not None:
            X_test_copy[num_columns] = scaler.transform(X_test_copy[num_columns])

        # Label encoding columns
        for col in cat_columns:
            # print(col)
            # Fitting the label Encoder
            le.fit(X_train_copy[col])
            # Handling unkown values
            for x in(X_val_copy, X_test_copy):
                if X_val_copy is not None:
                    unknown_values = list(set(X_val_copy[col].unique()) - set(le.classes_))
                    # print(unknown_values)
                    unk_replace = ['<unknown>'] * len(unknown_values)
                    X_val_copy[col] = X_val_copy[col].replace(unknown_values, unk_replace)                
                if X_test_copy is not None:
                    unknown_values = list(set(X_val_copy[col].unique()) - set(le.classes_))
                    # print(unknown_values)
                    unk_replace = ['<unknown>'] * len(unknown_values)
                    X_test_copy[col] = X_test_copy[col].replace(unknown_values, unk_replace)                
                    
            le.classes_ = np.append(le.classes_, '<unknown>')
            # print(le.classes_)
            # Transforming the categorical variable
            
            X_train_copy[col] = le.transform(X_train_copy[col])

            if X_val_copy is not None:
                X_val_copy[col] = le.transform(X_val_copy[col])
                                
            if X_test_copy is not None:
                X_test_copy[col] = le.transform(X_test_copy[col])    

        output = [X_train_copy]
        if X_val_copy is not None:
            output.append(X_val_copy)
                            
        if X_test_copy is not None:
            output.append(X_test_copy)     
        
        return tuple(output)

    else:
        name_to_config = {
            'config1_1':config1_1,
            'config1_2':config1_2,
            'config2_1':config2_1,
            'config2_2':config2_2,
            'config3_1':config3_1,
            'config3_2':config3_2,
        }
        requires_y = {
            'config1_1':False, 'config1_2':False, 
            'config2_1':True, 'config2_2':True,
            'config3_1':False, 'config3_2':False
        }
        config = name_to_config[config_name]
        output = []
        if requires_y[config_name]:
            if X_val_copy is not None:
                X_train_conf,X_val_conf = config(X_train_copy, Y_train, X_val_copy)
                output = [X_train_conf, X_val_conf]
            if X_test_copy is not None:
                X_train_conf,X_test_conf = config(X_train_copy, Y_train,X_test_copy)
                if output:
                    output.append(X_test_conf)
                else:
                    output = [X_train_conf, X_test_conf]
        else:
            if X_val_copy is not None:
                X_train_conf,X_val_conf = config(X_train_copy, X_val_copy)
                output = [X_train_conf, X_val_conf]
            if X_test_copy is not None:
                X_train_conf,X_test_conf = config(X_train_copy,X_test_copy)
                if output:
                    output.append(X_test_conf)
                else:
                    output = [X_train_conf, X_test_conf]
        return tuple(output)


    


In [47]:
# x1,x2=dataTransformer(X_train, X_val, X_test=None, Y_train=Y_train, config_name='config0')

In [48]:
# print(x1.describe())
# print(x2.head())

In [49]:
# x1,x2=dataTransformer(X_train, X_val, X_test=None, Y_train=Y_train, config_name='config1_1')

In [50]:
# print(x1.describe())
# print(x2.head())